In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('../data/lichess_db_puzzle.csv')

In [4]:
df.describe()

,Rating,RatingDeviation,Popularity,NbPlays
count,3.888765e+06,3.888765e+06,3.888765e+06,3.888765e+06
mean,1.515566e+03,9.027605e+01,8.354723e+01,1.674349e+03
std,5.432348e+02,4.303419e+01,2.140293e+01,4.459941e+03
min,3.990000e+02,4.900000e+01,-1.000000e+02,0.000000e+00
25%,1.085000e+03,7.500000e+01,8.200000e+01,9.300000e+01
50%,1.478000e+03,7.800000e+01,8.900000e+01,3.710000e+02
75%,1.917000e+03,8.800000e+01,9.300000e+01,1.325000e+03
max,3.331000e+03,5.000000e+02,1.000000e+02,1.007625e+06


In [5]:
lim=5000

In [6]:
df_train=df.iloc[:lim,:][['PuzzleId','FEN','Moves']]
targets=df.iloc[:lim,:]['Rating']

# Speed up

In [7]:
import chess
import pandas as pd
import numpy as np

def extract_features(fen):
    board = chess.Board(fen)
    features = {}

    # Material count
    piece_values = {chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3, chess.ROOK: 5, chess.QUEEN: 9}
    features['material_white'] = sum(len(board.pieces(piece, chess.WHITE)) * piece_values[piece] for piece in chess.PIECE_TYPES)
    features['material_black'] = sum(len(board.pieces(piece, chess.BLACK)) * piece_values[piece] for piece in chess.PIECE_TYPES)

    # Material imbalance
    features['material_imbalance'] = features['material_white'] - features['material_black']

    # King safety (number of attacking pieces)
    features['king_safety_white'] = sum(len(board.attackers(chess.BLACK, square)) for square in board.attackers(chess.BLACK, board.king(chess.WHITE)))
    features['king_safety_black'] = sum(len(board.attackers(chess.WHITE, square)) for square in board.attackers(chess.WHITE, board.king(chess.BLACK)))

    # Piece activity (mobility)
    features['mobility_white'] = sum(len(board.attacks(square)) for square in board.pieces(chess.KNIGHT, chess.WHITE))
    features['mobility_black'] = sum(len(board.attacks(square)) for square in board.pieces(chess.KNIGHT, chess.BLACK))

    # Pawn structure
    features['pawn_white'] = len(board.pieces(chess.PAWN, chess.WHITE))
    features['pawn_black'] = len(board.pieces(chess.PAWN, chess.BLACK))
    features['doubled_pawns_white'] = len([file for file in range(8) if len(board.pieces(chess.PAWN, chess.WHITE) & chess.SquareSet(chess.BB_FILES[file])) > 1])
    features['doubled_pawns_black'] = len([file for file in range(8) if len(board.pieces(chess.PAWN, chess.BLACK) & chess.SquareSet(chess.BB_FILES[file])) > 1])

    # Center control
    center_squares = [chess.D4, chess.E4, chess.D5, chess.E5]
    features['center_control_white'] = sum(1 for square in center_squares if board.piece_at(square) and board.piece_at(square).color == chess.WHITE)
    features['center_control_black'] = sum(1 for square in center_squares if board.piece_at(square) and board.piece_at(square).color == chess.BLACK)

    # Additional features can be added here...

    return pd.Series(features)


In [24]:
import pandas as pd
import chess

def extract_features(fen):
    board = chess.Board(fen)
    features = {}

    # Material count
    features['material_white'] = sum(len(board.pieces(piece, chess.WHITE)) for piece in chess.PIECE_TYPES)
    features['material_black'] = sum(len(board.pieces(piece, chess.BLACK)) for piece in chess.PIECE_TYPES)

    # Material imbalance
    features['material_imbalance'] = features['material_white'] - features['material_black']

    # King safety
    features['king_safety_white'] = sum(len(board.attackers(chess.BLACK, board.king(chess.WHITE))) for _ in range(1))
    features['king_safety_black'] = sum(len(board.attackers(chess.WHITE, board.king(chess.BLACK))) for _ in range(1))
    # Additional features can be added here...

    return pd.Series(features)

# Apply the function to your DataFrame
df_t = df_train['FEN'].apply(extract_features)



In [25]:
# Now each feature is a column in your DataFrame
df_t.sum()

material_white        4806383
material_black        4826493
material_imbalance     -20110
king_safety_white       29477
king_safety_black       33692
dtype: int64

In [29]:

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df_t, targets, test_size=0.2, random_state=42)

In [28]:
# Feature scaling
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 286923.92938682425
